# RNNs

Y'all punks can get the data from kaggle here: https://www.kaggle.com/uciml/sms-spam-collection-dataset/kernels 

In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/spam.csv', encoding = 'latin1')

In [2]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [6]:
from html import parser

# Drop the empty columns
df = df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)

# Rename the columns
df = df.rename(index=str, columns={"v1": "classification", "v2": "message"})

# Remove HTML encodings
df['message'] = df['message'].apply(parser.unescape)
df.head()

,classification,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [13]:
df['classification'] = pd.get_dummies(df['classification'])

In [14]:
print(df.head())

   classification                                            message
0               1  Go until jurong point, crazy.. Available only ...
1               1                      Ok lar... Joking wif u oni...
2               0  Free entry in 2 a wkly comp to win FA Cup fina...
3               1  U dun say so early hor... U c already then say...
4               1  Nah I don't think he goes to usf, he lives aro...


In [16]:
y = df['classification'].values
print(y)
X = df['message'].values
print(X)

[1 1 0 ..., 1 1 1]
[ 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'
 'Ok lar... Joking wif u oni...'
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"
 ..., 'Pity, * was in mood for that. So...any other suggestions?'
 "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free"
 'Rofl. Its true to its name']


We need to find how long our longest message is

In [25]:
max = 0
count = 0
for i in X:
    if len(i) > max:
        #if len(i) < 200:
        max = len(i)
        #else:
            #count +=1
        
print(max)
print(count)

910
0


In [29]:
from keras.preprocessing.text import Tokenizer
NUM_TOP_WORDS = None

tokenizer = Tokenizer(num_words=NUM_TOP_WORDS)
tokenizer.fit_on_texts(X)

X = tokenizer.texts_to_sequences(X)

In [46]:
import numpy as np
from keras.datasets import imdb
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)


# truncate and pad input sequences to be the same length
max_text_length = 910
X_train = sequence.pad_sequences(X_train, maxlen=max_text_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_text_length)

NUM_CLASSES = 1

In [47]:
from keras.models import Sequential, Input, Model
from keras.layers import Dense
from keras.layers import LSTM, GRU, SimpleRNN
from keras.layers.embeddings import Embedding

EMBED_SIZE = 50
rnns = []
input_holder = Input(shape=(X_train.shape[1], ))
shared_embed = Embedding(top_words, # input dimension (max int of OHE)
                EMBED_SIZE, # output dimension size
                input_length=max_text_length)(input_holder) # number of words in each sequence

for func in [SimpleRNN, LSTM, GRU]:
    
    x = func(50,dropout=0.2, recurrent_dropout=0.2)(shared_embed)
    x = Dense(NUM_CLASSES, activation='sigmoid')(x)
    rnn=Model(inputs=input_holder,outputs=x)
    rnn.compile(loss='binary_crossentropy', 
                  optimizer='rmsprop', 
                  metrics=['accuracy'])
    print(rnn.summary())
    rnns.append(rnn)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 910)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 910, 50)           250000    
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 50)                5050      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 255,101
Trainable params: 255,101
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 910)               0         
_________________________________________________________________

In [56]:
from keras.layers import Embedding
EMBED_SIZE = 100


word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, EMBED_SIZE))
embedding_layer = Embedding(len(word_index) + 1,
                            EMBED_SIZE,
                            weights=[embedding_matrix],
                            input_length=max_text_length,
                            trainable=False)

In [60]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

rnn = Sequential()
rnn.add(embedding_layer)
rnn.add(LSTM(100,dropout=0.2, recurrent_dropout=0.2))
rnn.add(Dense(NUM_CLASSES, activation='sigmoid'))
rnn.compile(loss='binary_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])
print(rnn.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 910, 100)          892000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 972,501
Trainable params: 80,501
Non-trainable params: 892,000
_________________________________________________________________
None


In [61]:
rnn.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Train on 4457 samples, validate on 1115 samples
Epoch 1/3
4457/4457 [==============================] - 268s - loss: 0.4261 - acc: 0.8555 - val_loss: 0.3970 - val_acc: 0.8682
Epoch 2/3
4457/4457 [==============================] - 274s - loss: 0.3996 - acc: 0.8654 - val_loss: 0.3903 - val_acc: 0.8682
Epoch 3/3
4457/4457 [==============================] - 260s - loss: 0.3969 - acc: 0.8654 - val_loss: 0.3901 - val_acc: 0.8682


In [50]:
print(X_train)

[[   0    0    0 ...,    6  130  269]
 [   0    0    0 ...,   91   30  312]
 [   0    0    0 ...,   30    8  440]
 ..., 
 [   0    0    0 ..., 3157    3  432]
 [   0    0    0 ...,   34 4666 1147]
 [   0    0    0 ...,   11   64 1743]]


In [51]:
print(y_train)

[1 1 1 ..., 1 1 1]
